# Create handle to workspace

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION = "545de186-b8b5-4f23-aba9-b7f0a043666d"
RESOURCE_GROUP = "ASDP"
WS_NAME = "azure-ml-workspace"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [2]:
# Verify that the handle works correctly.
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location, ":", ws.resource_group)

southeastasia : ASDP


# Access the registered data asset

In [4]:
# get a handle of the data asset and print the URI
credit_data = ml_client.data.get(name="kmp-legundi-dataset", version=1)
print(f"Data asset URI: {credit_data.path}")

Data asset URI: azureml://subscriptions/545de186-b8b5-4f23-aba9-b7f0a043666d/resourcegroups/ASDP/workspaces/azure-ml-workspace/datastores/workspaceblobstore/paths/UI/2025-02-12_063021_UTC/


# Create a job environment for pipeline steps

In [5]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [6]:
%%writefile {dependencies_dir}/conda.yaml
name: tensorflow-custom-new
channels:
  - conda-forge
  - anaconda
dependencies:
  - python=3.10
  - pip=22.3.1
  - pandas=2.1.2
  - numpy>=1.24
  - scipy=1.11.3
  - scikit-learn
  - pip:
      - tensorflow~=2.16.0
      - tensorflow-datasets~=4.9.0
      - matplotlib~=3.7.0
      - psutil~=5.9.0
      - tqdm~=4.66.3
      - ipykernel~=6.20.2
      - azureml-core==1.59.0
      - azureml-defaults==1.59.0
      - azureml-mlflow==1.59.0
      - azureml-telemetry==1.59.0
      - tensorboard~=2.16.0
      - optuna
      - debugpy~=1.6.3
      - cryptography>=41.0.4
      - idna>=3.7
      - azure-cosmos


Overwriting ./dependencies/conda.yaml


## Use the yaml file to create and register this custom environment in your workspace

In [7]:
from azure.ai.ml.entities import Environment

custom_env_name = "tensorflow-custom-new"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for asdp dataset pipeline",
    tags={"tensorflow": "2.16"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="0.1.0",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name tensorflow-custom-new is registered to workspace, the environment version is 0.1.0


# Create component 2: (using yaml definition)

## data_prep

In [8]:
import os

prep_src_dir = "./components/prep"
os.makedirs(prep_src_dir, exist_ok=True)

In [9]:
%%writefile {prep_src_dir}/data_prep.py

import os
import argparse
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

def main():
    # Argument parser
    parser = argparse.ArgumentParser(description="Data Preparation for One-Class SVM")
    parser.add_argument("--data", type=str, required=True, help="Path to the input CSV file or directory containing it")
    parser.add_argument("--test_train_ratio", type=float, default=0.2, help="Proportion of data used for testing")
    parser.add_argument("--feature_columns", type=str, required=True, help="Comma-separated list of feature column names")
    parser.add_argument("--timestamp_column", type=str, required=True, help="Name of the timestamp column in the dataset")
    parser.add_argument("--train_data", type=str, required=True, help="Path to save the training data folder")
    parser.add_argument("--test_data", type=str, required=True, help="Path to save the testing data folder")
    parser.add_argument("--missing_value_strategy", type=str, default="mean", choices=["mean", "median", "drop"], help="Strategy to handle missing values")
    args = parser.parse_args()

    # Handle input data path
    if os.path.isdir(args.data):
        csv_files = [f for f in os.listdir(args.data) if f.endswith(".csv")]
        if not csv_files:
            raise FileNotFoundError(f"No CSV file found in directory: {args.data}")
        csv_file_path = os.path.join(args.data, csv_files[0])
    else:
        csv_file_path = args.data

    # Load dataset
    print(f"Loading data from: {csv_file_path}")
    df = pd.read_csv(csv_file_path)
    print(f"Dataset loaded with shape: {df.shape}")

    # Validate timestamp column
    if args.timestamp_column not in df.columns:
        raise ValueError(f"Timestamp column '{args.timestamp_column}' not found in dataset")

    # Convert timestamp column to datetime
    print(f"Processing timestamp column: {args.timestamp_column}")
    timestamps = pd.to_datetime(df[args.timestamp_column]).to_numpy()

    # Select feature columns
    feature_columns = args.feature_columns.split(",")
    print(f"Using feature columns: {feature_columns}")
    X = df[feature_columns]

    # Handle missing values
    print("Checking and handling missing values...")
    if X.isnull().values.any():
        print(f"Missing values detected in feature columns: {X.isnull().sum()}")
        if args.missing_value_strategy == "mean":
            X = X.fillna(X.mean())
            print("Filled missing values with column mean.")
        elif args.missing_value_strategy == "median":
            X = X.fillna(X.median())
            print("Filled missing values with column median.")
        elif args.missing_value_strategy == "drop":
            X = X.dropna()
            timestamps = timestamps[X.index]  # Sinkronkan timestamps dengan data
            print("Dropped rows with missing values.")
    else:
        print("No missing values detected.")

    
    # Hitung indeks split berdasarkan 80% training dan 20% testing
    split_index = int(len(X) * 0.8)

    # Data training (80% pertama)
    X_train, timestamps_train = X[:split_index], timestamps[:split_index]

    # Data testing (20% terakhir)
    X_test, timestamps_test = X[split_index:], timestamps[split_index:]


    # Periksa panjang array setelah split
    print(f"Length of X_train: {len(X_train)}, timestamps_train: {len(timestamps_train)}")
    print(f"Length of X_test: {len(X_test)}, timestamps_test: {len(timestamps_test)}")

    # Normalisasi data menggunakan StandardScaler
    print("Normalizing data using StandardScaler...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Simpan scaler ke file agar dapat digunakan kembali
    scaler_path = os.path.join(args.train_data, "scaler.pkl")
    os.makedirs(args.train_data, exist_ok=True)
    joblib.dump(scaler, scaler_path)
    print(f"Scaler saved to: {scaler_path}")

    # Simpan train data (sudah dinormalisasi)
    os.makedirs(args.train_data, exist_ok=True)
    train_data_path = os.path.join(args.train_data, "X_train.npy")
    np.save(train_data_path, X_train_scaled)
    print(f"Training data saved to: {train_data_path}")

    # Simpan test data (sudah dinormalisasi)
    os.makedirs(args.test_data, exist_ok=True)
    test_data_path = os.path.join(args.test_data, "X_test.npy")
    np.save(test_data_path, X_test_scaled)
    print(f"Testing data saved to: {test_data_path}")
    
    # Simpan raw sensor values untuk train data (tidak dinormalisasi, untuk visualisasi)
    train_sensor_values_path = os.path.join(args.train_data, "sensor_values_train.npy")
    np.save(train_sensor_values_path, X_train.to_numpy())
    print(f"Train sensor values saved to: {train_sensor_values_path}")

    # Simpan timestamps untuk test data
    train_timestamps_path = os.path.join(args.train_data, "timestamps_train.npy")
    np.save(train_timestamps_path, timestamps_train)
    print(f"Test timestamps saved to: {train_timestamps_path}")

    # Simpan raw sensor values untuk test data (tidak dinormalisasi, untuk visualisasi)
    test_sensor_values_path = os.path.join(args.test_data, "sensor_values_test.npy")
    np.save(test_sensor_values_path, X_test.to_numpy())
    print(f"Test sensor values saved to: {test_sensor_values_path}")

    # Simpan timestamps untuk test data
    test_timestamps_path = os.path.join(args.test_data, "timestamps_test.npy")
    np.save(test_timestamps_path, timestamps_test)
    print(f"Test timestamps saved to: {test_timestamps_path}")

if __name__ == "__main__":
    main()


Overwriting ./components/prep/data_prep.py


In [10]:
%%writefile {prep_src_dir}/data_prep.yml
name: new_data_preparation_with_save_train_test_data
display_name: New Data Preparation with Save Train Test Data
type: command
version: 1.0
inputs:
  data:
    type: uri_file
    description: "Path to the input CSV file or directory containing it"
  test_train_ratio:
    type: number
    default: 0.2
    description: "Proportion of data used for testing"
  feature_columns:
    type: string
    description: "Comma-separated list of feature column names"
  timestamp_column:
    type: string
    description: "Name of the timestamp column in the dataset"
  missing_value_strategy:
    type: string
    default: "mean"
    description: "Strategy to handle missing values: 'mean', 'median', or 'drop'"

outputs:
  train_data:
    type: uri_folder
    description: "Output folder for training data (including timestamps_train.npy)"
  test_data:
    type: uri_folder
    description: "Output folder for testing data (including timestamps_test.npy)"

code: .
environment: azureml://registries/azureml/environments/sklearn-1.5/versions/18
command: >
  python data_prep.py
  --data ${{inputs.data}}
  --test_train_ratio ${{inputs.test_train_ratio}}
  --feature_columns ${{inputs.feature_columns}}
  --timestamp_column ${{inputs.timestamp_column}}
  --missing_value_strategy ${{inputs.missing_value_strategy}}
  --train_data ${{outputs.train_data}}
  --test_data ${{outputs.test_data}}


Overwriting ./components/prep/data_prep.yml


## registered component

In [11]:
from azure.ai.ml import load_component

# Load component from the YAML file
prep_component = load_component(source=os.path.join(prep_src_dir, "data_prep.yml"))

# Register the component
registered_component = ml_client.components.create_or_update(prep_component)

print(
    f"Component {registered_component.name} with Version {registered_component.version} is registered"
)


Uploading prep (0.01 MBs): 100%|██████████| 14295/14295 [00:00<00:00, 107884.91it/s]




Component new_data_preparation_with_save_train_test_data with Version 1.0 is registered


# train 

In [12]:
import os

train_src_dir = "./components/train"
os.makedirs(train_src_dir, exist_ok=True)

In [13]:
%%writefile {train_src_dir}/train_if.py

import os
import argparse
import numpy as np
import pandas as pd
import joblib
import mlflow
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import ParameterGrid

def train_isolation_forest(X_train, param_grid):
    """Hyperparameter Tuning with Grid Search."""
    best_model = None
    best_score = -np.inf
    best_params = None

    print("Starting Hyperparameter Tuning...")
    for params in ParameterGrid(param_grid):
        print(f"Testing parameters: {params}")
        
        model = IsolationForest(
            n_estimators=params["n_estimators"],
            contamination=params["contamination"],
            max_features=params["max_features"],
            random_state=42
        )
        model.fit(X_train)
        
        anomaly_scores = model.decision_function(X_train).mean()
        print(f"Mean Decision Score: {anomaly_scores}")

        if anomaly_scores > best_score:
            best_score = anomaly_scores
            best_model = model
            best_params = params

    print(f"Best Parameters: {best_params}, Best Score: {best_score}")
    return best_model, best_params

def main():
    parser = argparse.ArgumentParser(description="Train Isolation Forest Model with Hyperparameter Tuning")
    parser.add_argument("--train_data", type=str, required=True, help="Path to the training data folder")
    parser.add_argument("--test_data", type=str, required=True, help="Path to the testing data folder")
    parser.add_argument("--output_model", type=str, required=True, help="Path to save the trained model")
    parser.add_argument("--output_results", type=str, required=True, help="Path to save prediction results")
    parser.add_argument("--train_anomalies", type=str, required=True, help="Path to save detected training anomalies")
    args = parser.parse_args()

    mlflow.start_run()
    mlflow.sklearn.autolog()

    # Load training and testing data
    X_train = np.load(os.path.join(args.train_data, "X_train.npy"))
    X_test = np.load(os.path.join(args.test_data, "X_test.npy"))

    train_timestamps = np.load(os.path.join(args.train_data, "timestamps_train.npy"))
    train_sensor_values = np.load(os.path.join(args.train_data, "sensor_values_train.npy"))

    test_timestamps = np.load(os.path.join(args.test_data, "timestamps_test.npy"))
    test_sensor_values = np.load(os.path.join(args.test_data, "sensor_values_test.npy"))

    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")

    param_grid = {
        "n_estimators": [50, 100, 150],
        "contamination": [0.05, 0.1, 0.15],
        "max_features": [0.5, 0.75, 1.0]
    }

    best_model, best_params = train_isolation_forest(X_train, param_grid)

    # Save the best model locally
    os.makedirs(args.output_model, exist_ok=True)
    model_path = os.path.join(args.output_model, "isolation_forest_model.pkl")
    joblib.dump(best_model, model_path)
    print(f"Best Model saved to: {model_path}")

    # Register model via MLflow
    mlflow.sklearn.log_model(
        sk_model=best_model,
        registered_model_name="isolation_forest_model",
        artifact_path="isolation_forest"
    )

    print("Model registered successfully via MLflow!")

    # Detect and save anomalies in training data
    y_train_pred = np.where(best_model.predict(X_train) == -1, 1, 0)
    train_anomalies_df = pd.DataFrame({
        "timestamp": pd.to_datetime(train_timestamps),
        "sensor_value": train_sensor_values[:, 0],
        "is_anomaly": y_train_pred
    })
    os.makedirs(args.train_anomalies, exist_ok=True)
    train_anomalies_csv_path = os.path.join(args.train_anomalies, "train_anomalies.csv")
    train_anomalies_df.to_csv(train_anomalies_csv_path, index=False)
    print(f"Training anomalies saved to: {train_anomalies_csv_path}")

    # Save test predictions
    y_test_pred = np.where(best_model.predict(X_test) == -1, 1, 0)
    results_df = pd.DataFrame({
        "timestamp": pd.to_datetime(test_timestamps),
        "sensor_value": test_sensor_values[:, 0],
        "is_anomaly": y_test_pred
    })
    results_csv_path = os.path.join(args.output_results, "test_predictions.csv")
    results_df.to_csv(results_csv_path, index=False)
    print(f"Test predictions saved to: {results_csv_path}")

    mlflow.end_run()

if __name__ == "__main__":
    main()


Overwriting ./components/train/train_if.py


In [14]:
%%writefile {train_src_dir}/train_if.yml

name: new_isolation_forest_training_with_hypertuning
display_name: New Isolation Forest Model Training with Hyperparameter Tuning
type: command
version: 1.0
inputs:
  train_data:
    type: uri_folder
    description: "Path to folder containing training data (NumPy files)."
  test_data:
    type: uri_folder
    description: "Path to folder containing testing data (NumPy files)."
outputs:
  output_model:
    type: uri_folder
    description: "Folder containing the trained model (as .pkl file)."
  output_results:
    type: uri_folder
    description: "Folder containing prediction results as CSV."
  train_anomalies:
    type: uri_folder
    description: "Folder containing anomalies detected in training data."
code: .
environment: azureml://registries/azureml/environments/sklearn-1.5/versions/18
command: >
  python train_if.py
  --train_data ${{inputs.train_data}}
  --test_data ${{inputs.test_data}}
  --output_model ${{outputs.output_model}}
  --output_results ${{outputs.output_results}}
  --train_anomalies ${{outputs.train_anomalies}}


Overwriting ./components/train/train_if.yml


In [15]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source=os.path.join(train_src_dir, "train_if.yml"))

# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

Uploading train (0.01 MBs): 100%|██████████| 5715/5715 [00:00<00:00, 102559.65it/s]




Component new_isolation_forest_training_with_hypertuning with Version 1.0 is registered


# Create the pipeline from component

In [16]:
from azure.ai.ml import dsl, Input

@dsl.pipeline(
    compute="serverless",
    description="E2E pipeline for Isolation Forest training with hyperparameter tuning",
)
def isolation_forest_pipeline(
    pipeline_job_data_input: Input(type="uri_folder"),
    pipeline_job_test_data: Input(type="uri_folder"),
    pipeline_job_feature_columns: str,
    pipeline_job_timestamp_column: str,
):
    # Step 1: Data Preparation
    data_prep_job = prep_component(
        data=pipeline_job_data_input,
        feature_columns=pipeline_job_feature_columns,
        timestamp_column=pipeline_job_timestamp_column
    )

    # Step 2: Model Training using train_component (Isolation Forest)
    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=pipeline_job_test_data
    )

    # Return pipeline outputs from train_component
    return {
        "pipeline_job_output_model": train_job.outputs.output_model,
        "pipeline_job_output_results": train_job.outputs.output_results,
        "pipeline_job_train_anomalies": train_job.outputs.train_anomalies,
    }


In [17]:
pipeline = isolation_forest_pipeline(
    pipeline_job_data_input=Input(type="uri_folder", path="path/to/train_data"),
    pipeline_job_test_data=Input(type="uri_folder", path="path/to/test_data"),
    pipeline_job_feature_columns="pressure_lo1",  # Contoh feature columns
    pipeline_job_timestamp_column="Timestamp"  # Nama kolom timestamp
)

# Submit the pipeline
pipeline_job = ml_client.jobs.create_or_update(pipeline)
ml_client.jobs.stream(pipeline_job.name)


Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


MlException: 
[37m
[30m
1) One or more files or folders do not exist.
[39m[39m

Details: 

[31m(x) No such file or directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mycompute-ml/code/Users/eni.sawitri/Isolation Forest Model/path/to/train_data[39m

Resolutions: 
1) Double-check the directory path you provided and enter the correct path.
If using the CLI, you can also check the full log in debug mode for more details by adding --debug to the end of your command

Additional Resources: The easiest way to author a yaml specification file is using IntelliSense and auto-completion Azure ML VS code extension provides: [36mhttps://code.visualstudio.com/docs/datascience/azure-machine-learning.[39m To set up VS Code, visit [36mhttps://docs.microsoft.com/azure/machine-learning/how-to-setup-vs-code[39m


In [ ]:
https://azuremlworkspa8827795289.blob.core.windows.net/azureml/ExperimentRun/dcid.468c6fcd-337c-4000-a305-75a4b61997ce/model/model.pkl

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION = "545de186-b8b5-4f23-aba9-b7f0a043666d"
RESOURCE_GROUP = "ASDP"
WS_NAME = "azure-ml-workspace"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)


# # Register model
# model = Model(
#     path="azureml://jobs/ce4ffa02-04c3-443d-b0c3-1cc7301de730/outputs/output_model/isolation_forest_model.pkl",
#     name="isolation_forest_model",
#     description="Isolation Forest model for anomaly detection"
# )

# ml_client.models.create_or_update(model)

# Register model
model = Model(
    path="https://azuremlworkspa8827795289.blob.core.windows.net/azureml/ExperimentRun/dcid.468c6fcd-337c-4000-a305-75a4b61997ce/model/model.pkl",
    name="isolation_forest_model",
    description="Isolation Forest model for anomaly detection"
)

ml_client.models.create_or_update(model)



HttpResponseError: (UserError) The request is invalid.
Code: UserError
Message: The request is invalid.
Exception Details:	(InvalidUrl) The URL 'https://azuremlworkspa8827795289.blob.core.windows.net/azureml/ExperimentRun/dcid.468c6fcd-337c-4000-a305-75a4b61997ce/model/model.pkl' is invalid
	Code: InvalidUrl
	Message: The URL 'https://azuremlworkspa8827795289.blob.core.windows.net/azureml/ExperimentRun/dcid.468c6fcd-337c-4000-a305-75a4b61997ce/model/model.pkl' is invalid
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "13e4781e76392863894ab95242a09150",
        "request": "65d14c33d756f67b"
    }
}Type: Environment
Info: {
    "value": "southeastasia"
}Type: Location
Info: {
    "value": "southeastasia"
}Type: Time
Info: {
    "value": "2025-02-13T07:08:22.954898+00:00"
}

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration
)
from azure.identity import DefaultAzureCredential

In [2]:
# Define an endpoint name
endpoint_name = "isolation-forest2"

# Example way to define a random name
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="this is a sample endpoint",
    auth_mode="key"
)

In [ ]:
model = Model(path="azureml://jobs/ce4ffa02-04c3-443d-b0c3-1cc7301de730/outputs/output_model/isolation_forest_model.pkl")
env = Environment(
    conda_file="../model-1/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../model-1/onlinescoring", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)